<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js037_viscositySPH2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js037_viscositySPH2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.09.26 created,  last updated on 2025.08.28
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js037_viscositySPH2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js037_viscositySPH2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.06.14 created, last updated on 2018.11.18
//    ver 0.0.1  2019.01.16 v1, last updated on 2021.05.28
//    ver 0.0.2  2021.10.31 v2, last updated on 2021.10.31
//    ver 0.0.3  2023.03.22 v3, last updated on 2023.08.22
//
// --------------------  smoothed particle hydrodynamics 2D
//
// - particle base Lagrangian method
//    Monaghan; "Smoothed Particle Hydrodynamics"
//              Annu. Rev. Astron. Astrophys.1992. 30:543-74
//
//    W(x-xi,h) : kernel weight function (q=|x-xi|/h)
//      = aKernel*(1-1.5*q^2+0.75*q^3)  (q<1)
//      = aKernel*0.25*(2-q)^3  (1<=q<2)
//      = 0  (q>=2)
//
//        aKernel = 2/3/h (1D)
//                = 10/(7Pi)/h^2 (2D)
//                = 1/Pi/h^3 (3D)
//
//    f(x) --> sum[mj/rhoj*f(xj)*W(x-xj,h), j]
//    grad f(x) --> -sum[mj/rhoj*f(xj)*grad W(x-xj,h), j]
//
// - time step
//   (1) registration - set section[][][]
//   (2) set density[]
//   (3) set pressure[]
//   (4) Verlet step1 (t = t + dt/2)
//   (5) set acceleration ax[] ay[] and power[] <-- x(t+dt/2),y(t+dt/2)
//   (6) Verlet step2 (t = (t + dt/2) + dt/2)
//   (7) set boundary
//   goto (1)
//
// - for fast calculation - O(N) calculation
//     set section: smoothed particle divided into lattice section (see setSection() )
//     calculate acceleration: sum up near section (see accCalc() )
//       near means r<2*hh0
//
// - artificial viscosity
//     dv_a/dt = -sum(P_b/rho_b^2 + P_a/rho_a^2 + pai_ab) grad W_ab
//       pai_ab = (-alpha c_ab mue_ab + beta mue_ab^2)/rho_ab (v_ab*r_ab<0) or 0 (v_ab*r_ab>0)
//         mue_ab = (h v_ab*r_ab)/(r_ab^2+etha^2), etha^2 nearly= 0.01*hh0*hh0
//
// --------------------
*/

const viscositySPH2D = (function(){ // ====================  viscositySPH2D Module  ====================

	let g_NNp = 1600;						// number of smoothed particles
	let g_Nsx = 30;							// number of division to x-lattice section
	let g_Nsy = 30;							// number of division to y-lattice section
	let g_xMax = 10.0;						// (m) x-Box size
	let g_yMax = 10.0;						// (m) y-Box size
	let g_sysTime = 0.0;					// (s) system time
	let g_dt = 0.0001;						// (s) time step
	let g_hh0 = 0.25;						// (m) diameter of smoothed particles
	let g_cViscosity = 10.0;				// for air viscosity, alpha=1,beta=2 in this case
	let g_thermalDiffusivity = 0.1;			// (m^2/s) thermal diffusivity (in 3D case for air , 3.4e-5)
	let g_gravity = 0.0;					// (m/s^2) gravity acceleration eg. 9.8;

	const g_xx = dim1( g_NNp );				// (m) smooth particle pos-x
	const g_yy = dim1( g_NNp );				// (m) smooth particle pos-y
	const g_vx = dim1( g_NNp );				// (m/s) smooth particle velocity-x
	const g_vy = dim1( g_NNp );				// (m/s) smooth particle velocity-y
	const g_ax = dim1( g_NNp );				// (m/s^2) smooth particle total accelaration-x
	const g_ay = dim1( g_NNp );				// (m/s^2) smooth particle total accelaration-y
	const g_hh = dim1( g_NNp );				// (m) smooth particle diameter
	const g_mass = dim1( g_NNp );			// (kg) smooth particle mass
	const g_cp = dim1( g_NNp );				// (J/(kg*K)) smooth particle specific heat
	const g_density = dim1( g_NNp );		// (kg/m^3) smooth particle density
	const g_pressure = dim1( g_NNp );		// (N/m^2) smooth particle pressure
	const g_energy = dim1( g_NNp );			// (J) smooth particle energy = mass*cp*temp
	const g_power = dim1( g_NNp );			// (W) smooth particle power : energy(t+dt) = energy(t)+power*dt
	const g_section = dimInt3( g_Nsx, g_Nsy, 100 ); // registration: particles in Nsx x Nsy sections

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] =  new Int32Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	// public
	function setInitialCondition() {
		const nnp=g_NNp;
		g_sysTime = 0.0;
		for (let i=0; i<nnp; i++) {
			g_mass[i] = (0.029/0.0224)/4.0; //air
			g_cp[i] = 1000.0; //air
			g_xx[i] = 0.1*(i%40)+1.0;
			g_yy[i] = 0.1*Math.floor(i/40)+2.0;
			g_vx[i] = 0.0;
			g_vy[i] = 0.0;
			g_ax[i] = 0.0;
			g_ay[i] = 0.0;
			g_hh[i] = g_hh0;
		}
		setDensity();
		for (let i=0; i<nnp; i++) {
			g_energy[i] = g_mass[i]*g_cp[i]*300; //300=temp(K)
			g_power[i] = 0.0;
		}
	}


	// --------------------  time evolution  --------------------

	// public
	function timeEvolution(nCalc) {
		for (let i=0; i<nCalc; i++) {
			timeStep();
			g_sysTime += g_dt;
		}
	}

	function timeStep() {
		const nnp=g_NNp, dtv2 = g_dt/2.0;
		// (1) registration - set section[][][]
		setSection();
		// (2) set density
		setDensity();
		// (3) set pressure
		for (let i=0; i<nnp; i++) {
			g_pressure[i] = 8.31*g_density[i]*(g_energy[i]/(g_mass[i]*g_cp[i]));
		}
		// (4) Verlet step1 (t = t + dt/2)
		for (let i=0; i<nnp; i++) {
			g_vx[i] += dtv2*g_ax[i];
			g_vy[i] += dtv2*g_ay[i];
			g_xx[i] += g_vx[i]*g_dt;
			g_yy[i] += g_vy[i]*g_dt;
		}
		// (5) set acceleration ax[] ay[] and power[]
		accCalc();
		// (6) Verlet step2 (t = t + dt/2 + dt/2)
		for (let i=0; i<nnp; i++) {
			g_vx[i] += dtv2*g_ax[i];
			g_vy[i] += dtv2*g_ay[i];
			g_energy[i] += g_power[i]*g_dt;
		}
		// (7) boundary
		const rr = 1.0;
		for (let i=0; i<nnp; i++) {
			if (g_xx[i] < 0.0) {
				g_xx[i] = 0.0; g_vx[i] = -rr*g_vx[i]; g_vy[i] = rr*g_vy[i];
			}
			if (g_xx[i] > g_xMax) {
				g_xx[i] = g_xMax; g_vx[i] = -rr*g_vx[i]; g_vy[i] = rr*g_vy[i];
			}
			if (g_yy[i] < 0.0) {
				g_yy[i] = 0.0; g_vx[i] = rr*g_vx[i]; g_vy[i] = -rr*g_vy[i];
			}
			if (g_yy[i] > g_yMax) {
				g_yy[i] = g_yMax; g_vx[i] = rr*g_vx[i]; g_vy[i] = -rr*g_vy[i];
			}
		}
	}

	// --- (1) registration

	function setSection() {
		const nnp=g_NNp, nsx=g_Nsx, nsy=g_Nsy;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				g_section[i][j][0] = 0;
			}
		}
		for (let ip=0; ip<nnp; ip++) {
			let i = Math.floor(nsx*g_xx[ip]/g_xMax); if (i>=nsx) i = nsx-1;
			let j = Math.floor(nsy*g_yy[ip]/g_yMax); if (j>=nsy) j = nsy-1;
			const iq = g_section[i][j][0]+1;
			g_section[i][j][0] = iq;
			g_section[i][j][iq] = ip;
		}
	}

	function maxSection() {
		const nsx=g_Nsx, nsy=g_Nsy;
		let m=0;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				if (g_section[i][j][0]>m) m = g_section[i][j][0];
			}
		}
		return m;
	}

	// --- (2) set density[]

	function setDensity() {
		const nnp=g_NNp, nsx=g_Nsx, nsy=g_Nsy;
		const rcc = g_hh0*2.0;
		const rcc2 = rcc*rcc;
		for (let i=0; i<nnp; i++) {
			let s = 0.0;
			let i0 = Math.floor(nsx*(g_xx[i]-rcc)/g_xMax); if (i0<0) i0 = 0;
			let i1 = Math.floor(nsx*(g_xx[i]+rcc)/g_xMax); if (i1>=nsx) i1 = nsx-1;
			let j0 = Math.floor(nsy*(g_yy[i]-rcc)/g_yMax); if (j0<0) j0 = 0;
			let j1 = Math.floor(nsy*(g_yy[i]+rcc)/g_yMax); if (j1>=nsy) j1 = nsy-1;
			for (let ii=i0; ii<=i1; ii++) {
				for (let jj=j0; jj<=j1; jj++) {
					for (let iq=1; iq<=g_section[ii][jj][0]; iq++) {
						const j = g_section[ii][jj][iq];
						const r2=(g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]);
						if (r2<rcc2) {
							const rij = Math.sqrt(r2);
							s += g_mass[j]*kernel(rij, (g_hh[i]+g_hh[j])/2.0);
						}
					}
				}
			}
			g_density[i] = s;
		}
	}

	// --- (5) set acceleration ax[] ay[] and power[] + viscosity

	function accCalc() {
		const nnp=g_NNp, nsx=g_Nsx, nsy=g_Nsy;
		const alpha=1.0, beta=2.0, cv=g_cViscosity; // for viscosity
		const rcc = g_hh0*2.0;
		const rcc2 = rcc*rcc;
		for (let i=0; i<nnp; i++) {
			g_ax[i] = 0.0; g_ay[i] = g_gravity;
			boundaryForce(i, g_hh0, 1000000.0);
			g_power[i] = 0.0;
			const ai = g_pressure[i]/(g_density[i]*g_density[i]);
			let i0 = Math.floor(nsx*(g_xx[i]-rcc)/g_xMax); if (i0<0) i0 = 0;
			let i1 = Math.floor(nsx*(g_xx[i]+rcc)/g_xMax); if (i1>=nsx) i1 = nsx-1;
			let j0 = Math.floor(nsy*(g_yy[i]-rcc)/g_yMax); if (j0<0) j0 = 0;
			let j1 = Math.floor(nsy*(g_yy[i]+rcc)/g_yMax); if (j1>=nsy) j1 = nsy-1;
			for (let ii=i0; ii<=i1; ii++) {
				for (let jj=j0; jj<=j1; jj++) {
					for (let iq=1; iq<=g_section[ii][jj][0]; iq++) {
						const j = g_section[ii][jj][iq];
						const r2 = (g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]);
						if (r2<rcc2) {
							const rij = Math.sqrt(r2);
							if (rij>0) {
								const aj = g_pressure[j]/(g_density[j]*g_density[j]);
								const vr = (g_xx[i]-g_xx[j])*(g_vx[i]-g_vx[j])+(g_yy[i]-g_yy[j])*(g_vy[i]-g_vy[j]); //
								let paiij = 0; //
								if (vr<0.0) { // for viscosity
									const mu = 0.5*vr/(r2+0.01*g_hh0*g_hh0);
									paiij = (-alpha*cv*mu+beta*mu*mu)/((g_density[i]+g_density[j])/2.0);
								}
								const b = g_mass[j]*(ai+aj+paiij);
								const gradW = dwwvdr(rij, (g_hh[i]+g_hh[j])/2.0);
								const gradWx = gradW*(g_xx[i]-g_xx[j])/rij;
								const gradWy = gradW*(g_yy[i]-g_yy[j])/rij;
								g_ax[i] += -b*gradWx;
								g_ay[i] += -b*gradWy;
								const lapW = d2wwvdr2(rij, (g_hh[i]+g_hh[j])/2.0);
								g_power[i] += 0.5*b*((g_vx[i]-g_vx[j])*gradWx+(g_vy[i]-g_vy[j])*gradWy) +
									g_thermalDiffusivity*g_mass[j]/g_density[j]*(g_energy[j]-g_energy[i])*lapW;
							}
						}
					}
				}
			}
		}
	}

	function boundaryForce(i, rc, k) {
		let r;
		if (g_xx[i] < rc) {
			r = rc-g_xx[i];
			g_ax[i] += k*r*r;
		}
		if (g_xx[i] > g_xMax-rc) {
			r = g_xx[i]-(g_xMax-rc);
			g_ax[i] += -k*r*r;
		}
		if (g_yy[i] < rc) {
			r = rc-g_yy[i];
			g_ay[i] += k*r*r;
		}
		if (g_yy[i] > g_yMax-rc) {
			r = g_yy[i]-(g_yMax-rc);
			g_ay[i] += -k*r*r;
		}
	}

	// --- smooth particle

	function distance(i,j) {
		const x = g_xx[i]-g_xx[j], y = g_yy[i]-g_yy[j];
		return Math.sqrt(x*x+y*y);
	}

	function kernel(r,h) {
		let ret = 0.0;
		const q = r/h;
		const a = 10.0/(7.0*3.141592)/(h*h);
		if (q<1.0) {
			ret = a*(1.0-1.5*q*q+0.75*q*q*q);
		} else if (q<2.0) {
			ret = a*0.25*(2.0-q)*(2.0-q)*(2.0-q);
		}
		return ret;
	}

	function dwwvdr(r,h) {
		let ret = 0.0;
		const q = r/h;
		const a = 10.0/(7.0*3.141592)/(h*h*h);
		if (q<1.0) {
			ret = a*(-3.0*q+2.25*q*q);
		} else if (q<=2.0) {
			ret = -a*0.75*(2.0-q)*(2.0-q);
		}
		return ret;
	}

	function d2wwvdr2(r, h) {
		let ret = 0.0;
		const q = r/h;
		const a = 10.0/(7.0*3.141592)/(h*h*h*h);
		if (q<1.0) {
			ret = a*(-3.0+4.5*q);
		} else if (q<=2.0) {
			ret = a*0.75*(2.0-q);
		}
		return ret;
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition()
		evolve:			timeEvolution,			// timeEvolution( nCalc )

		getSysParam:	function() { return [ g_xMax, g_yMax, g_dt, g_hh0, g_NNp ]; },
		getSysTime:		function() { return g_sysTime; },
		getSPData:		function(i) { return [ g_hh[i], g_mass[i], g_cp[i] ]; },
		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getAcc:			function(i) { return [ g_ax[i], g_ay[i] ]; },
		getDensity:		function(i) { return g_density[i]; },
		getPressure:	function(i) { return g_pressure[i]; },
		getTemp:		function(i) { return (g_energy[i]/(g_mass[i]*g_cp[i])); },
		getEnergy:		function(i) { return g_energy[i]; },
		getPower:		function(i) { return g_power[i]; },
	};

})(); // ====================  viscositySPH2D end  ====================


const js037 = (function(){ // ====================  js Module  ====================

	const theModule = viscositySPH2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 0;  // no use
	let v_nCalc = 2;

	let p_xMax, p_yMax, p_dt, p_hh0, p_NNp; // = theModule.getSysParam();
	let sysTime;
	let nowData = [];
  let xxList = [];
	let yyList = [];
	let vxList = [];
	let vyList = [];
  let densList = [];
	let pressList = [];
	let tempList = [];

	let dispMode = 2;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;

	let breakFlag = false;
	let perticleFlag = true;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( );
			[ p_xMax, p_yMax, p_dt, p_hh0, p_NNp ] = theModule.getSysParam();
			perticleFlag = true;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( 1 );
		}

		draw( ctx, dispMode );

		if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

	function setParticlesData() {
		nowData = [ sysTime, p_NNp ];
    for (let i=0; i<p_NNp; i++) {
			let x, y, vx, vy;
			[ x, y ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
			[ vx, vy ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
			densList[i] = theModule.getDensity(i);
			pressList[i] = theModule.getPressure(i);
			tempList[i] = theModule.getTemp(i);
		}
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const xp = 40, yp = 15, xSize = 400, ySize = 400, scale = xSize/p_xMax;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		if ( dispMode<=3 ) {
			drawField( ctx, dispMode, p_NNp, p_hh0, scale, xp, yp );
		} else if ( dispMode==4 ) {
			drawFlow( ctx, p_NNp, scale, xp, yp );
		}

		// caption
		sysTime = theModule.getSysTime();
		ctx.fillStyle = "#888888";
		ctx.fillText(`Box = ${p_xMax.toFixed(1)}x${p_yMax.toFixed(1)} (m)`, xp, yCanvasSize-30);
		ctx.fillText(`N = ${p_NNp}`, xp+260, yCanvasSize-30);
		ctx.fillText(`time = ${sysTime.toFixed(3)} (s)`, xp, yCanvasSize-10);
		//document.getElementById("text_caption").innerHTML = msg;
	}

	function drawField( ctx, dispMode, nnp, hh0, sc, xp, yp ) {
		const yMax = p_yMax, twoPi = 2.0*Math.PI, dmag=5.0, pmag=0.003, tmag=2.0;

		for (let i=0; i<nnp; i++) {
			let xi, yi, hue;
			[ xi, yi ] = theModule.getPosition(i);
			if (dispMode==0) { // smooth particle
				hue = 180;
			} else if (dispMode==1) { // density
				const dens = theModule.getDensity(i);
				hue = Math.floor(18180-dens*dmag) % 360;
			} else if (dispMode==2) { // pressure
				const press = theModule.getPressure(i);
				hue = Math.floor(18180-press*pmag) % 360;
			}  else if (dispMode==3) { // temp
				const temp = theModule.getTemp(i)-273;
				hue = Math.floor(18240-temp*tmag) % 360;
			}
			ctx.strokeStyle = `hsl(${hue},100%,50%)`;
			ctx.beginPath();
			ctx.arc(xi*sc+xp, (yMax-yi)*sc+yp, hh0*sc, 0, twoPi, false);
			ctx.stroke();
		}
	}

	function drawFlow( ctx, nnp, sc, xp, yp ) {
		const yMax = p_yMax, vmag = 80.0*p_dt*sc;

		for (let i=0; i<nnp; i++) {
			let xi, yi, vxi, vyi;
			[ xi, yi ] = theModule.getPosition(i);
			[ vxi, vyi ] = theModule.getVelocity(i);
			const x = xi*sc+xp, y = (yMax-yi)*sc+yp;
			const hue = Math.floor((Math.atan2(vyi,vxi))*180.0/Math.PI+180.0);
			ctx.strokeStyle = `hsl(${hue},100%,50%)`;
			ctx.beginPath();
			ctx.moveTo( x, y );
			ctx.lineTo( x+vxi*vmag, y-vyi*vmag);
			ctx.stroke();
		}
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		v_nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, p_NNp ];
  }

  function pygetParticlesList() {
		perticleFlag = false;
    return [ nowData, xxList, yyList, vxList, vyList, densList, pressList, tempList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, nnp ]
    pygetParticlesList, pygetParticlesList, //() :return [ nowData, xxList, yyList, vxList, vyList, densList, pressList, tempList ]
	};

})(); // ====================  js037 module end  ====================


const js = js037;
//window.addEventListener('load', js.main );
js.main();

// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js037] viscosity - smoothed particle hydrodynamics 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme: viscosity</label>
    <span style="margin-right: 150px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label> disp. mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>smooth particle</option><option selected>density</option>
<option selected>pressure</option><option>temperature</option>
<option>flow</option>
</select>
    <span style="margin-right: 80px;"></span>
<label>speed(nCalc/frame):</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option>1</option><option selected>2</option>
<option>3</option><option>4</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# simulator run
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, nnp ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f"i = {i:>2d},  time = {sysTime:>8.3f}")
  time.sleep(1)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# change dispMode

import time

dispModeList = [ '0: smooth particle', '1: density', '2: pressuree', '3: temperature', '4: flow' ]

# simulator run
exec_html_js()
print("-- start --")

# change dispMode
for dispMode in [ 0, 1, 2, 3, 4 ]:
  # change dispMode
  eval_js( 'js.pysetDispMode({})'.format(dispMode) )
  print( "-- dispMode: ", dispModeList[dispMode])
   # get data and print
  [ sysTime, nnp ] = eval_js( 'js.pygetData({})'.format(0) )
  print(f"\t time = {sysTime:>8.3f}")
  time.sleep(2)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# get data and save

import time
import numpy as np

# simulator run
exec_html_js()
print("-- start --")

# wait 7 sec
time.sleep(12)

# get particle dara and plot
print("-- get particles data --")
[ nowData, xxList, yyList, vxList, vyList, densList, pressList, tempList ] = eval_js( 'js.pygetParticlesList()' )
[ gotTime, nnp ] = nowData
print(f"number of smoothed particle = {nnp} at time ={gotTime:>6.3f}")

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

# pack data
packed_data = np.array([ xxList, yyList, vxList, vyList, densList, pressList, tempList ])
print(f'shape of packed_data :{packed_data.shape}')

# save data
np.save( 'js037_data.npy', packed_data )
print("-- saved --")

In [ ]:
# load data and set numpy array

import numpy as np

x_box_size = 10.0
y_box_size = 10.0

# load data
loaded_data = np.load('js037_data.npy')
print(f'the shape of loaded_data :{loaded_data.shape}')

# set numpy array
X = loaded_data[0]     # x-position of SPH particle
Y = loaded_data[1]     # y-position of SPH particle
U = loaded_data[2]     # x-velocity of SPH particle
V = loaded_data[3]     # x-velocity of SPH particle
Dens = loaded_data[4]  # x-density at the particle position
Press = loaded_data[5] # x-pressure at the particle position
Temp = loaded_data[6]  # x-temperature at the particle position
print(f'the shape of X, Y, U, V, Dens, Press, Temp :{X.shape}')
Vabs = np.sqrt(U**2 + V**2)

In [ ]:
# plot pressure

import matplotlib.pyplot as plt

print("-- plot pressure --")
plt.figure(figsize=(6, 6))
plt.title("pressure(x,y)")
plt.scatter(X, Y, s=100, c=Press, cmap='jet')
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)
plt.show()

In [ ]:
# colab AI wrote:
# prompt: plot Dens, Temp in 1 x 2 sub plot

print("-- plot Dens, Temp in 1 x 2 sub plot --")
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("density(x,y)")
plt.scatter(X, Y, s=100, c=Dens, cmap='jet')
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)

plt.subplot(1, 2, 2)
plt.title("temperature(x,y)")
plt.scatter(X, Y, s=100, c=Temp, cmap='jet')
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)

plt.show()


In [ ]:
# pressure(x,y) scatter 3D plot

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 3d scatter plot
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Press, s=50, c=Press, cmap='jet')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('Pressure')
plt.title('Pressure(x,y) in 3D')
plt.show()

In [ ]:
# pressure(x,y) scatter 3D plot - plotly

import numpy as np
import plotly.graph_objects as go

# 3d scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=X, y=Y, z=Press,
    mode='markers', marker=dict(size=5, color=Press, colorscale='jet' ))])

fig.update_layout(
    title='Pressure(x,y) in 3D',
    width=800, height=700,
    scene = dict(
        zaxis_title='Pressure',
        camera=dict(
            eye=dict(x=-0.5, y=-1.5, z=1.0),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=0, z=1)  # upside vector
    )),)

fig.show()

# save figure as html file
#fig.write_html("js037_pressure.html")

In [ ]:
# pressure(x,y) Mesh 3d plot - plotly

import plotly.graph_objects as go

# mesh 3d plot pressure(x,y)
fig = go.Figure(data=[go.Mesh3d(
    x=X, y=Y, z=Press,
    opacity=0.5,
  )])

fig.update_layout(
    width=800, height=700,
    scene = dict(
        zaxis_title='Pressure',
        camera=dict(
            eye=dict(x=-0.5, y=-1.5, z=1.0),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=0, z=1)  # upside vector
    )),)

fig.show()

In [ ]:
# flow(x,y) vector plot

import numpy as np
import matplotlib.pyplot as plt

# quiver plot of flow(x,y)
plt.figure(figsize=(6, 6))
plt.title("flow(x,y) ")
plt.quiver(X, Y, U, V, Vabs, cmap='jet')
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)
plt.show()

In [ ]:
# plot flow at particle position
# colab AI wrote:
# prompt: plot flow(x,y) as plot line (x,y) - (x + u*dt, y + v*dt) at every particle (x,y) position

import numpy as np
import matplotlib.pyplot as plt

# plot flow(x,y) as plot line (x,y) - (x + u*dt, y + v*dt) at every particle (x,y) position

# set dt
dt = 0.001 * 5 # time span

# create plot
plt.figure(figsize=(6, 6))
plt.title("Flow")

# plot particle position
plt.scatter(X, Y, s=5, c='blue', alpha=0.5)

# plot flow line
for i in range(len(X)):
  plt.plot([X[i], X[i] + U[i] * dt], [Y[i], Y[i] + V[i] * dt], color='red', alpha=0.5)

# set axis and show plot
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)
plt.xlabel("x")
plt.ylabel("y")
plt.show()


In [ ]:
# colab AI wrote:
# prompt: corr and scatter matrix of X, Y, U, V, Vabs, Temp with pandas

import pandas as pd

# create a DataFrame
df = pd.DataFrame({'X': X, 'Y': Y, 'U': U, 'V': V, 'Vabs': Vabs, 'Temp': Temp})

# calculate the correlation matrix
corr_matrix = df.corr()

# print the correlation matrix
print("Correlation matrix:")
print(corr_matrix)

# create a scatter matrix
scatter_matrix = pd.plotting.scatter_matrix(df)

# display the scatter matrix
plt.show()


In [ ]:
# prompt: contour plot3D from X, Y, and Press with plotly
# Gemini wrote:
# pressure(x,y) Contour plot in 3D - plotly

import plotly.graph_objects as go
import numpy as np

# create a grid of x, y values
x_grid = np.linspace(X.min(), X.max(), 50)
y_grid = np.linspace(Y.min(), Y.max(), 50)
X_mesh, Y_mesh = np.meshgrid(x_grid, y_grid)

# interpolate pressure onto the grid (using a simple nearest neighbor for demonstration)
from scipy.interpolate import griddata
Press_mesh = griddata((X, Y), Press, (X_mesh, Y_mesh), method='nearest')

# 3d contour plot
fig = go.Figure(data=[go.Surface(
    x=X_mesh, y=Y_mesh, z=Press_mesh,
    colorscale='jet',
  )])

fig.update_layout(
    title='Pressure(x,y) Contour Plot in 3D',
    width=800, height=800,
    scene = dict(
        zaxis_title='Pressure',
        camera=dict(
            eye=dict(x=0.5, y=-1.5, z=1.0),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=0, z=1)  # upside vector
    )),)

fig.show()